# 🎀 Analysis of Female-Led ("Girly") TV Shows on Netflix

This notebook focuses on identifying and analyzing female-led TV shows on Netflix
using descriptive data analysis techniques.

## 1. Dataset Structure

In this step, we examine the overall structure of the dataset,
including its size, columns, and data types.

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use("default")

In [15]:
df = pd.read_csv("../data/raw/netflix_titles.csv")

## 1. Dataset Size

Before starting the analysis, we check the number of rows and columns
to understand the scale of the dataset.

In [16]:
df.shape

(8807, 12)

## 2. Column Overview

In this step, we list all columns in the dataset
to understand what information is available for analysis.

In [17]:
df.columns

Index(['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'duration', 'listed_in', 'description'],
      dtype='object')

## 3. Data Types and Missing Values

In this section, we inspect data types and identify missing values
to assess data quality before deeper analysis.

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


## 4. Missing Values Overview

This step shows the exact number of missing values in each column,
allowing us to identify problematic fields.

In [19]:
df.isnull().sum().sort_values(ascending=False)

director        2634
country          831
cast             825
date_added        10
rating             4
duration           3
show_id            0
type               0
title              0
release_year       0
listed_in          0
description        0
dtype: int64

## 5. Handling Missing Cast Information

Since identifying female-led TV shows requires cast information,
titles with missing cast data are excluded from further analysis.

This is a conscious analytical decision to ensure accuracy,
rather than making assumptions or imputations.

In [20]:
df_clean= df.dropna(subset=["cast"])

In [21]:
df_clean.shape

(7982, 12)

## 6. Filtering Only TV Shows

To keep the analysis focused and consistent,
we limit the dataset to TV Shows only.
Movies are excluded since character-centric analysis
is more meaningful for series.

In [22]:
df_tv = df_clean[df_clean["type"] == "TV Show"]

In [23]:
df_tv.shape

(2326, 12)

## 7. Defining Female-Led TV Shows

Since the dataset does not explicitly indicate the gender of main characters,
we define a TV show as *female-led* if the first listed cast member
has a female first name.

This heuristic provides a simple and transparent approximation
suitable for an exploratory analysis.

In [28]:
df_tv = df_clean[df_clean["type"] == "TV Show"].copy()

In [29]:
df_tv["main_actor"] = df_tv["cast"].str.split(",").str[0]

In [30]:
df_tv[["cast", "main_actor"]].head()

,cast,main_actor
1,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",Ama Qamata
2,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",Sami Bouajila
4,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",Mayur More
5,"Kate Siegel, Zach Gilford, Hamish Linklater, H...",Kate Siegel
8,"Mel Giedroyc, Sue Perkins, Mary Berry, Paul Ho...",Mel Giedroyc
